In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [3]:
from langchain.chat_models import init_chat_model
model = init_chat_model(model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        model_provider="groq",
                        temperature = 0,
                        max_retries=3,
                        timeout=60,
                        max_tokens=300
                        )

In [4]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model=model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model=model,
    tools=[square]
)

## Calling subagents

In [7]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model=model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [8]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [9]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='8b44c554-e874-48f3-85fc-b95314db9c8c'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'en9fm1k0q', 'function': {'arguments': '{"x":456}', 'name': 'call_subagent_1'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 793, 'total_tokens': 825, 'completion_time': 0.058137687, 'completion_tokens_details': None, 'prompt_time': 0.017533567, 'prompt_tokens_details': None, 'queue_time': 0.049298673, 'total_time': 0.075671254}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3bf0-0649-7561-bcc1-267e787d741e-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'en9fm1k0q', 'type': 'tool_call'}], usage_metadata={